In [51]:
#Un scrapper de pdf. Par exemple sur un site hébergant une multitude de pdf sous forme de liens dans une <div> spécifique.
#L'idéal est d'utiliser la console de votre navigateur (Ctrl+Maj+I) pour identifier des motifs récurents dans le code html.  

from urllib.parse import urlparse
import scrapy
from scrapy.http import Request

class recup(scrapy.Spider):
    name = "recup"

# choix du domaine à parser 
    allowed_domains = ["www.domaine.com"]
# page à parser
    start_urls = ["http://www.domaine.com.html"]
    
# récupère les liens de type div h3 (à changer pour cibler d'autre liens)
    def parse(self, response):
        for href in response.css('div#all_results h3 a::attr(href)').extract():
            yield Request(
                url=response.urljoin(href),
                callback=self.parse_article
            )
# pour tout lien ayant une extension en .pdf lance la fontion save_pdf
    def parse_article(self, response):
        for href in response.css('div.download_wrapper a[href$=".pdf"]::attr(href)').extract():
            yield Request(
                url=response.urljoin(href),
                callback=self.save_pdf
            )

# sauvegarde les pdf
    def save_pdf(self, response):
        path = response.url.split('/')[-1]
        self.logger.info('Saving PDF %s', path)
        with open(path, 'wb') as f:
            f.write(response.body)
            

            
            